# HELP

https://practicaldatascience.co.uk/machine-learning/how-to-use-category-encoders-to-transform-categorical-variables
https://xgboost.readthedocs.io/en/latest/parameter.html

## Load modules and packages

In [2]:
import numpy as np
import pandas as pd
import datetime
from tqdm.notebook import tqdm
from random import sample
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn import tree
from sklearn.metrics import roc_auc_score
import random
from sklearn.pipeline import Pipeline
import category_encoders as ce
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.experimental import enable_halving_search_cv # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingRandomSearchCV

## Load data

In [3]:
random.seed(10) 

from functions import *
# Load data - set index column, decimal point, separator
data = pd.read_csv('hw1_devsample.csv', sep=',',
                   decimal='.', index_col='SK_ID_CURR')

# print time of data being loaded - use strftime
print(f'Data loaded on:   {datetime.datetime.now().strftime(format="%Y-%m-%d %H:%M:%S")}')
data_xgb = data.copy()

data_test = pd.read_csv('hw1_outofsample.csv', sep=',',decimal='.', index_col='SK_ID_CURR')

Data loaded on:   2022-05-06 11:12:01


## Small changes

In [4]:
target = data_xgb['TARGET']
data_xgb = data_xgb.drop(['MONTH','TIME','DAY','BASE','TARGET'], axis=1)
data_xgb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80000 entries, 215138 to 259380
Columns: 191 entries, NAME_CONTRACT_TYPE to LAST_TRANSACTION_TIME_MONTHS
dtypes: float64(138), int64(37), object(16)
memory usage: 117.2+ MB


In [5]:
cols_pred = data_xgb.columns
cols_pred_cat = [col for col in cols_pred if data_xgb[col].dtype == 'O']
cols_pred_cat

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

## Split data train and test

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_xgb, target, test_size=0.2, random_state=123)

# Test manually

### Test target encoder

In [7]:
target_encoder = ce.target_encoder.TargetEncoder(verbose=0, 
cols=cols_pred_cat,
#drop_invariant=False, 
return_df=True, handle_missing='value', 
handle_unknown='value',
#, min_samples_leaf=1
smoothing=1.0
)

In [8]:
X_train = target_encoder.fit_transform(X_train, y_train)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64000 entries, 147643 to 291895
Columns: 191 entries, NAME_CONTRACT_TYPE to LAST_TRANSACTION_TIME_MONTHS
dtypes: float64(154), int64(37)
memory usage: 93.8 MB


In [9]:
# find columns with infinity values
cols_with_inf = []
for col in X_train.columns:
    if np.any(np.isinf(X_train[col])):
        cols_with_inf.append(col)
        print(f'Column {col} includes infinity values.')

# find columns with negative infinity values
cols_with_neginf = []
for col in X_train.columns:
    if np.any(np.isneginf(X_train[col])):
        cols_with_neginf.append(col)
        print(f'Column {col} includes negative infinity values.')

for col in cols_with_inf:
    X_train[col].replace(np.inf, 9999999, inplace = True)

Column MEAN_AMTCR_1M_3M_DIV_MEAN_AMTCR_3M_12M includes infinity values.
Column MEAN_AMTCR_1M_3M_TYPE_EQ_ACTIVE_DIV_MEAN_AMTCR_3M_12M_TYPE_EQ_ACTIVE includes infinity values.
Column MEAN_AMTCR_1M_3M_TYPE_EQ_CLOSED_DIV_MEAN_AMTCR_3M_12M_TYPE_EQ_CLOSED includes infinity values.
Column MEAN_AMTCR_OVERDUE_0M_INFM_DIV_MEAN_AMTCR_0M_INFM includes infinity values.
Column MEAN_AMTCR_OVERDUE_0M_INFM_TYPE_EQ_ACTIVE_DIV_MEAN_AMTCR_0M_INFM_TYPE_EQ_ACTIVE includes infinity values.


### Test manually XGBOOST

In [10]:
model = xgb.XGBClassifier(objective='binary:logistic', booster='gbtree', max_depth = 4, eta = 0.4, gamma = 3)

In [11]:
print(model)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.4, eval_metric=None, gamma=3,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=None, ...)


In [12]:
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False, eta=0.4,
              eval_metric=None, gamma=3, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.400000006, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, ...)

In [13]:
X_test = target_encoder.fit_transform(X_test, y_test)
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16000 entries, 224276 to 135830
Columns: 191 entries, NAME_CONTRACT_TYPE to LAST_TRANSACTION_TIME_MONTHS
dtypes: float64(154), int64(37)
memory usage: 23.4 MB


In [14]:
pred_test = model.predict(X_test)

In [15]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, pred_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 92.17%


# Pipeline

In [16]:
target_encoder = ce.target_encoder.TargetEncoder( 
cols=cols_pred_cat,
#drop_invariant=False, 
return_df=True, handle_missing='value', 
handle_unknown='value'
#, min_samples_leaf=1
#, smoothing=1.0
)

In [17]:
model = xgb.XGBClassifier(objective='binary:logistic', booster='gbtree')

In [18]:
pipe = Pipeline([
    ('Mean_Target_Encoding', target_encoder),
    ('model', model)
])

## Halving Random Search CV

In [19]:
param_grid = {
    'Mean_Target_Encoding__smoothing': [0.5, 1.0, 1.5,2.0],
    'model__max_depth': [2, 4, 6, 8],
    'model__eta' : [0.2,0.3,0.4],
    'model__gamma': [0,1,2,3,4,5]
}


## Apply pipeline to data

In [20]:
search = HalvingRandomSearchCV(pipe, param_grid, cv = 3,factor=5,random_state=0).fit(X_train, y_train)

c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 288 is smaller than n_iter=5333. Running 288 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [21]:
search.best_params_ 

{'model__max_depth': 2,
 'model__gamma': 4,
 'model__eta': 0.4,
 'Mean_Target_Encoding__smoothing': 1.0}

{'model__max_depth': 2,
 'model__gamma': 4,
 'model__eta': 0.4,
 'Mean_Target_Encoding__smoothing': 1.0}

In [31]:
search.best_score_

0.9118837675350702

In [35]:
pred_test = search.predict_proba(X_test)

## Calculate AUC

In [37]:
pred_test = search.predict(X_test)

In [39]:
sum(pred_test)

33

In [41]:
sum(y_test)

1247.0

In [40]:
accuracy = accuracy_score(y_test, pred_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 92.17%


In [27]:
print('AUC',roc_auc_score(y_test, pred_test))

AUC 0.5049695355071925


# Predict on the second dataset

In [ ]:
prediction = ...

In [ ]:
data_saving = pd.DataFrame({'SK_ID_CURR' : data_test.index ,'prediction' : prediction})
data_saving.to_csv('DS2_22_HW1_CADIOU&PAIN.csv',index=False,sep=',')
data_saving.info()